# Manipulando dados da b3

 ## Importando libs

In [45]:
import os
import pandas as pd
import chardet

## Extraindo dados

In [39]:
def read_files(path,name_file, year_date, type_file):
    """
    Lê um arquivo de texto fixo com colunas de dados financeiros e retorna um DataFrame.

    Args:
        path (str): Caminho do diretório onde o arquivo está localizado.
        name_file (str): Nome base do arquivo.
        year_date (int or str): Ano ou data usada para compor o nome do arquivo.
        type_file (str): Extensão do arquivo (ex: 'txt').

    Returns:
        pd.DataFrame: DataFrame contendo os dados lidos do arquivo com colunas pré-definidas.

    Colunas:
        - 'data_pregao': Data do pregão.
        - 'codbdi': Código BDI (mercado de ações).
        - 'sigla_acao': Sigla da ação.
        - 'nome_acao': Nome da ação.
        - 'preco_abertura': Preço de abertura.
        - 'preco_maximo': Preço máximo.
        - 'preco_minimo': Preço mínimo.
        - 'preco_fechamento': Preço de fechamento.
        - 'qtd_negocios': Quantidade de negócios.
        - 'volume_negocios': Volume de negócios.

    Observação:
        Utiliza o método `pd.read_fwf` para ler o arquivo com colunas de largura fixa (`colspecs`).
    """
    
    _file = f"{path}{name_file}{year_date}.{type_file}"

    colspecs = [
        (2,10),
        (10,12),
        (12,24),
        (27,39),
        (56,69),
        (69,82),
        (82,95),
        (108,121),
        (152,170),
        (170,188)
    ]
    
    names = ['data_pregao','codbdi','sigla_acao','nome_acao','preco_abertura','preco_maximo','preco_minimo','preco_fechamento','qtd_negocios','volume_negocios']
    df = pd.read_fwf(_file,colspecs = colspecs, names = names, skiprows = 1, encoding='latin-1')

    return df

## Filtrando lote padrão

In [40]:
def filter_stocks(df):
    """
    Filtra o DataFrame para manter apenas ações ordinárias (código BDI igual a 2).

    Args:
        df (pd.DataFrame): DataFrame contendo os dados financeiros.

    Returns:
        pd.DataFrame: DataFrame filtrado contendo apenas as ações ordinárias.
    
    Operações:
        - Remove linhas onde o código BDI não é igual a 2.
        - Remove a coluna 'codbdi' do DataFrame.
    """
    df = df[df['codbdi']==2]
    df = df.drop(['codbdi'], axis=1)
    return df

## Ajustando o campo das datas

In [41]:
def parse_date(df):
    """
    Converte a coluna 'data_pregao' para o formato datetime.

    Args:
        df (pd.DataFrame): DataFrame contendo a coluna 'data_pregao'.

    Returns:
        pd.DataFrame: DataFrame com a coluna 'data_pregao' convertida para formato datetime.

    Operações:
        - Converte os valores da coluna 'data_pregao' para o formato de data (YYYYMMDD).
        - Utiliza a função `pd.to_datetime` com tratamento de erros.
    """
    
    df['data_pregao'] = pd.to_datetime(df['data_pregao'], format= '%Y%m%d',errors='coerce')
    return df

## Ajustando os campos numericos

In [42]:
def parse_values(df):
    """
    Converte os valores das colunas de preços de centavos para reais.

    Args:
        df (pd.DataFrame): DataFrame contendo os preços de abertura, máximo, mínimo e fechamento.

    Returns:
        pd.DataFrame: DataFrame com os valores das colunas de preços ajustados.

    Operações:
        - Divide os valores das colunas de preço por 100 para convertê-los de centavos para reais.
        - Converte os valores para o tipo float.
    """

    df['preco_abertura'] = (df['preco_abertura'] /100).astype(float)
    df['preco_maximo'] = (df['preco_maximo'] /100).astype(float)
    df['preco_minimo'] = (df['preco_minimo'] /100).astype(float)
    df['preco_fechamento'] = (df['preco_fechamento'] /100).astype(float)
    return df

## Juntando arquivos

In [43]:
def concat_files(path, name_file, year_date, type_file, final_file):
    """
    Concatena e processa arquivos de dados, salvando o resultado final em um arquivo CSV.

    Args:
        path (str): Caminho do diretório onde os arquivos estão localizados.
        name_file (str): Nome base dos arquivos a serem lidos.
        year_date (list): Lista de anos para identificar os arquivos a serem lidos.
        type_file (str): Extensão ou tipo dos arquivos a serem lidos.
        final_file (str): Nome do arquivo final onde o resultado será salvo.

    Funções internas:
        read_files: Lê os arquivos com base nos parâmetros fornecidos.
        filter_stocks: Aplica filtros relacionados a ações no DataFrame.
        parse_date: Processa e formata as colunas de datas.
        parse_values: Processa e formata os valores no DataFrame.

    Processamento:
        - Para cada ano na lista `year_date`, o arquivo correspondente é lido e filtrado.
        - O DataFrame resultante é concatenado ao DataFrame final.
        - O DataFrame final é salvo em um arquivo CSV no diretório especificado.

    Resultado:
        Um arquivo CSV contendo a concatenação e o processamento dos dados de todos os anos especificados.
    """
    
    df_final = pd.DataFrame()
    for i, y in enumerate(year_date):
        df = read_files(path, name_file, y, type_file)
        df = filter_stocks(df)
        df = parse_date(df)
        df = parse_values(df)

        df_final = pd.concat([df_final,df], ignore_index=True)

    output_file = os.path.join(path, final_file)
    df_final.to_csv(output_file, index=False)

## Executando programa etl

In [44]:
year_date = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']
path = 'C://Users//user//projects//b3//'
name_file = 'COTAHIST_A'
type_file = 'txt'
final_file = 'all_bovespa(1).csv'
concat_files(path, name_file, year_date, type_file, final_file)